# <center>Tabular Playground Series - May/2021<center>
## <center>Neural Network with Keras<center>
---
    
As I recently finished Kaggle’s Intro to Deep Learning course, I decided to start practicing on this month’s tabular playground competition. This is my first notebook on Neural Networks outside the exercises from Kaggle’s course. If you spot any mistakes or have any tips to help me building some ‘intuition’ on the subject, feel free to share.
 
Based on [@subinium](https://www.kaggle.com/subinium) [great notebook for beginners](https://www.kaggle.com/subinium/tps-may-deeplearning-pipeline-for-beginner), I created a similar model and tweaked the number of nodes on hidden layers, dropout rate and the batch size, observing the train vs validation loss plot. Then, I added the early stopping procedure to halt the training and get the best weights.
    
Update: First, I’ve applied log transformation and standard scaling to the dataset and reduced the number of units in each hidden layer of the NN. Then I changed the batch size, the learning rate and included the callback ‘ReduceLROnPlateau’, as suggested by [@pourchot](https://www.kaggle.com/pourchot) and used in [his notebook](https://www.kaggle.com/pourchot/lb-1-0896-keras-nn-with-20-folds). Public LB score before these changes: 1.09559
    
    
My other notebooks in this competition:
- [Tabular Playground Series - May/2021: LightGBM Tuned with Hyperopt](https://www.kaggle.com/jonaspalucibarbosa/tps05-21-lgbm-tuned-w-hyperopt)
- [Tabular Playground Series - May/2021: Model Stacking using Logistic Regression as Meta-Learner](https://www.kaggle.com/jonaspalucibarbosa/tps05-21-model-stacking-meta-learner-lr)
    

## Importing Libraries and Datasets

In [ ]:
import pandas as pd       
import matplotlib as mat
import matplotlib.pyplot as plt    
import numpy as np
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import callbacks

In [ ]:
#Trying to get reproducible results
from numpy.random import seed
seed(42)
from tensorflow.random import set_seed
set_seed(42)

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv', index_col = 'id')
Y_train = df_train['target'].copy()
X_train = df_train.copy().drop('target', axis = 1)

X_test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv', index_col = 'id')

In [ ]:
#Update: Applied Log Transformation and Standard Scaling
scaler = StandardScaler()

X_concat = pd.concat([X_train, X_test]).reset_index(drop=True)
for col in X_train.columns:
    X_concat[col] = np.log(X_concat[col]-(min(X_concat[col]-1)))
X_concat = pd.DataFrame(scaler.fit_transform(X_concat), columns = X_train.columns)
X_concat

In [ ]:
X_train_logscaled = X_concat[:X_train.shape[0]][X_train.columns]
X_test_logscaled = X_concat[X_train.shape[0]:][X_train.columns].reset_index(drop=True)

X_train_logscaled

In [ ]:
X_test_logscaled

In [ ]:
class_map = {'Class_1': 0,
            'Class_2': 1,
            'Class_3': 2,
            'Class_4': 3}
Y_train = Y_train.map(class_map).astype('int')
Y_train

In [ ]:
#Converting target series to matrix for multiclass classification on Keras

Y_train = to_categorical(Y_train)
Y_train

## Creating and Evaluating the NN

In [ ]:
#Included a BatchNormalization on the beginning
def get_model():
    model = keras.Sequential([
        layers.BatchNormalization(input_shape = [50]),
        layers.Dense(units = 128, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(units = 64, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(units = 32, activation = 'relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),    
        layers.Dense(4, activation = 'softmax'),
    ])
    
    return model

In [ ]:
keras.backend.clear_session()

model = get_model()
model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=0.002), metrics='accuracy')

model.summary()

In [ ]:
X_train_split, X_val_split, Y_train_split, Y_val_split = train_test_split(X_train_logscaled, Y_train, test_size = 0.2, random_state = 42
                                                    , stratify = Y_train)

In [ ]:
early_stopping = callbacks.EarlyStopping(
    patience=25,
    min_delta=0.0000001,
    restore_best_weights=True,
)

#New callback
plateau = callbacks.ReduceLROnPlateau(
    factor = 0.5,                                     
    patience = 2,                                   
    min_delt = 0.0000001,                                
    cooldown = 0,                               
    verbose = 1
) 

In [ ]:
history = model.fit(X_train_split, Y_train_split,
          batch_size = 256, epochs = 100,
          validation_data=(X_val_split, Y_val_split),
          callbacks=[early_stopping, plateau]);

In [ ]:
score = model.evaluate(X_val_split, Y_val_split, verbose = 0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}%'.format(score[1] * 100))

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
sns.lineplot(x = history.epoch, y = history.history['loss'])
sns.lineplot(x = history.epoch, y = history.history['val_loss'])
ax.set_title('Learning Curve (Loss)')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.legend(['train', 'test'], loc='best')
plt.show()

## Making Predictions

In [ ]:
Y_train = df_train['target'].copy()
Y_train = Y_train.map(class_map).astype('int')
Y_train

In [ ]:
def prediction (X_train, Y_train, X_test):
    
    keras.backend.clear_session()

    kfold = StratifiedKFold(n_splits = 10)

    y_pred = np.zeros((50000,4))
    train_oof = np.zeros((100000,4))
    
    for idx in kfold.split(X=X_train, y=Y_train):
        train_idx, val_idx = idx[0], idx[1]
        xtrain = X_train.iloc[train_idx]
        ytrain = Y_train.iloc[train_idx]
        xval = X_train.iloc[val_idx]
        yval = Y_train.iloc[val_idx]
        
        ytrain = to_categorical(ytrain)
        yval = to_categorical(yval)
        
        # fit model for current fold
        model = get_model()
        model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=0.002), metrics='accuracy')
        
        model.fit(xtrain, ytrain,
        batch_size = 256, epochs = 100,
        validation_data=(xval, yval),
        callbacks=[early_stopping, plateau]);

        #create predictions
        y_pred += model.predict(X_test)/kfold.n_splits
        print(y_pred)
               
        val_pred = model.predict(xval)
        # getting out-of-fold predictions on training set
        train_oof[val_idx] = val_pred
        
        # calculate and append logloss
        fold_logloss = metrics.log_loss(yval,val_pred)
        print("Logloss: {0:0.5f}". format(fold_logloss))
  
    return y_pred, train_oof

In [ ]:
nn_pred, train_oof = prediction (X_train_logscaled, Y_train, X_test_logscaled)

In [ ]:
print("Logloss: {0:0.6f}".format(metrics.log_loss(Y_train,train_oof)))

In [ ]:
train_oof = pd.DataFrame(train_oof, columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4'])
train_oof

In [ ]:
pred_test = pd.DataFrame(nn_pred, columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4'])
pred_test

In [ ]:
train_oof.to_csv('nn_train_oof.csv', index=False)

train_oof

In [ ]:
output = pred_test
output['id'] = X_test.index
output.to_csv('submission.csv', index=False)

output